# Transformer
完全基于注意力机制，没有任何CNN layer或RNN layer

## 1. 模型
### 1.1 编码器（encoder）
多个相同层汇聚而成，每个层都有两个子层（sublayer）。
- 第一个子层是多头自注意力（multihead self-attention）汇聚；
- 第二个子层是基于位置的前馈神经网络（positionwise feed-forward network）。
- 在计算自注意力时，查询、键和值都来自前一个编码器**层**的输出。
- 每个子层使用残差连接（residual connection）。
- 对于序列中任意位置的任何输入$\mathbf{x}\in\mathbb{R}^d$，都要求$\mathrm{sublayer}(\mathbf{x})\in\mathbb{R}^d$，以便残差连接满足$\mathbf{x}+\mathrm{sublayer}(\mathbf{x})\in\mathbb{R}^d$
- 在残差连接加法后，紧接着应用层规范化（layer normalization）。
- 因此对于输入序列对应的每个位置，transformer编码器都将输出一个$d$维表示向量
### 1.2 解码器（decoder）
多个相同层叠加而成。每层有三个子层（sublayer）
- 第一个子层是

In [17]:
import math
import pandas
import torch
from torch import nn
from d2l import torch as d2l

## 2. Positionwise FFN
基于位置的前馈网络中对序列中的所有位置的表示进行变换时使用的是一个MLP。
- 输入X（批量大小，时间步数或序列长度，隐藏单元数或特征维度）
- 输出：被两层MLP转换为（批量大小，时间步数，ffn_num_outputs）

In [18]:
class PositionWiseFNN(nn.Module):
    '''基于位置的前馈神经网络'''
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs, **kwargs):
        super(PositionWiseFNN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)
        
    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

In [19]:
'''运行例子'''
ffn = PositionWiseFNN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 8])

## 3. Residual Connection and Layer Normalization
残差连接后紧跟层规范化组成。层规范化是基于特征维度进行规范化，在自然语言处理任务中，批量规范化通常不如层规范化效果好。

In [20]:
class AddNorm(nn.Module):
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)
    
    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

In [21]:
'''运行例子'''
add_norm = AddNorm([3, 4], dropout=0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

## 4. Encoder
先实现一个层，包含两个字层：Multihead Attention + PositionWiseFNN，都使用了ResidualConnection和LayerNormalization

In [22]:
class EncoderBlock(nn.Module):
    '''编码器块'''
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, use_bias = False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(key_size, query_size, value_size, num_hiddens,
                                                num_heads, dropout, use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFNN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)
        
    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))


In [23]:
'''运行例子'''
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

torch.Size([2, 100, 24])

In [24]:
class TransformerEncoder(d2l.Encoder):
    def __init__(self, vocab_size, key_size, query_size, value_size,
                 num_hiddens, norm_shape, ffn_num_input, ffn_num_hiddens,
                 num_heads, num_layers, dropout, use_bias=False, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(vocab_size, num_hiddens)
        self.pos_encoding = d2l.PositionalEncoding(num_hiddens, dropout)
        self.blks = nn.Sequential()
        for _ in range(num_layers):
            self.blks.add(
                EncoderBlock(num_hiddens, ffn_num_hiddens, num_heads, dropout, use_bias)
            )
    def forward(self, X, valid_lens, *args):
        # 因为位置编码值-1和1之间，
        # 因此嵌入值乘以嵌入维度的平方根进行缩放
        # 然后再与位置编码相加。
        X = self.attention_weights = [None] * len(self.blks)
        for i, blk in enumerate(self.blks):
            X = blk(X, valid_lens)
            self.attention_weights[i] = blk.attention.attention_weights
        return X

## 5. Decoder
多个相同层组成，在DecoderBlock类中实现每个层包含三个子层：
解码器自注意力、编码器-解码器注意力、position wise FNN。也需要Residual Connection和Layer Normalization。

In [ ]:
class DecoderBlock(nn.Module):
    '''解码器块'''
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                 dropout, i, **kwargs):
        super(DecoderBlock, self).__init__(**kwargs)
        self.i = i
        self.attention1 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout
        )
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.attention2 = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout
        )
        self.addnorm2 = AddNorm(norm_shape, dropout)
        self.fnn = PositionWiseFNN(ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm3 = AddNorm(norm_shape, dropout)
        
    def forward(self, X, state):
        enc_outputs, enc_valid_len = state[0], state[1]
        # 训练阶段，输出序列所有词元都在同一时间处理
        # 因此sate[2][self.i]初始化为None
        # 预测阶段，输出序列通过词元一个接一个编码
        # 因此sate[2][self.i]包含着直到当前时间步的第i个块编码的输出表示
        if state[2][self.i] is None:
            key_values = X
        return